#### Link of the Pycaret docs : https://pycaret.readthedocs.io/en/latest/api/time_series.html

In [11]:
#importing necessary packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import numpy as np 
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import imageio
import os
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit
from pycaret.time_series import TSForecastingExperiment
import pmdarima as pm


In [12]:
#função para tratar campo data
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('dbsoja_br.xls', sheet_name='Planilha1')
print(df.head(10))

   date  producao  produtividade     area
0  1977   12145.0           1748   6949.0
1  1978    9726.0           1250   7780.0
2  1979   10200.0           1251   8151.0
3  1980   14887.4           1700   8755.9
4  1981   15484.8           1781   8693.4
5  1982   12890.9           1536   8393.2
6  1983   14532.9           1728   8412.0
7  1984   15340.5           1674   9162.9
8  1985   18211.5           1808  10074.0
9  1986   13207.5           1369   9644.4


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           48 non-null     int64  
 1   producao       48 non-null     float64
 2   produtividade  48 non-null     int64  
 3   area           48 non-null     float64
dtypes: float64(2), int64(2)
memory usage: 1.6 KB


### Avaliando a produção de soja

In [14]:
df.drop(['produtividade','area'], axis = 1,inplace=True)
df.head(10)


,date,producao
0,1977,12145.0
1,1978,9726.0
2,1979,10200.0
3,1980,14887.4
4,1981,15484.8
5,1982,12890.9
6,1983,14532.9
7,1984,15340.5
8,1985,18211.5
9,1986,13207.5


In [15]:
df.set_index('date',inplace=True)

In [16]:
df=df.sort_index(axis=0)
print(df.head(10))

      producao
date          
1977   12145.0
1978    9726.0
1979   10200.0
1980   14887.4
1981   15484.8
1982   12890.9
1983   14532.9
1984   15340.5
1985   18211.5
1986   13207.5


fh: Optional[int or list or np.array], default = 1

The forecast horizon to be used for forecasting. Default is set to 1 i.e. forecast one point ahead. When integer is passed it means N continuous points in the future without any gap. If you want to forecast values with gaps, you must pass an array e.g. np.arange([13, 25]) will skip the first 12 future points and forecast from the 13th point till the 24th point ahead (note in numpy right value is inclusive and left is exclusive).

In [17]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=42, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,42
1,Target,producao
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(48, 1)"
5,Transformed data shape,"(48, 1)"
6,Transformed train set shape,"(47, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [18]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

## Checking Train-Test Split

In [19]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

## Comparing Models

The most basic assumption of time series forecasting is that past patterns continue into the future. 
Based on this assumption, in the short-term future forecast, since the environment in which the data is generated is similar to the present, 
the uncertainty is small and the forecast can be well-predicted. However, as we move towards long-term future forecasting, the possibility of the environment in which data is generated is likely to change, and unpredictable situations occur, and as errors continue to accumulate, uncertainty about the forecast data is bound to increase.

We compare the performance of various models and try to select the 4 models with the best performance.

In [20]:
top4 = exp.compare_models(n_select = 4)

## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data. 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

## Difference Plot Using Order

In [21]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":50})

### auto_arima

In [22]:
auto_arima = exp.create_model('auto_arima')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = auto_arima, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(auto_arima, fh = 8)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,0.2979,0.2251,1346.7888,1346.7888,0.0097,0.0097
1,2021.0000,4.1415,3.1179,19667.6901,19667.6901,0.1567,0.1453
2,2022.0000,4.8165,3.6297,23845.8135,23845.8135,0.1542,0.1671
Mean,nan,3.0853,2.3243,14953.4308,14953.4308,0.1068,0.1074
SD,nan,1.9902,1.4990,9771.3769,9771.3769,0.0687,0.0696


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2024,146878.4794
2025,156542.0370
2026,162440.5723
2027,168339.1077
2028,174237.6431
2029,180136.1785
2030,186034.7139
2031,191933.2492


###  gbr_cds_dt

In [ ]:
gbr_cds_dt = exp.create_model('gbr_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = gbr_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 6},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,2.0444,1.5449,9243.1299,9243.1299,0.0663,0.0686
1,2021.0000,0.4886,0.3678,2320.2926,2320.2926,0.0185,0.0183
2,2022.0000,5.7204,4.3109,28320.7566,28320.7566,0.1832,0.2016
Mean,nan,2.7511,2.0745,13294.7264,13294.7264,0.0893,0.0962
SD,nan,2.1935,1.6527,10994.4716,10994.4716,0.0692,0.0773


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## ets

In [ ]:
ets = exp.create_model('ets')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 6},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,1.3250,1.0012,5990.3987,5990.3987,0.0430,0.0439
1,2021.0000,3.8529,2.9006,18296.9663,18296.9663,0.1457,0.1358
2,2022.0000,3.1650,2.3851,15669.3768,15669.3768,0.1013,0.1068
Mean,nan,2.7809,2.0957,13318.9139,13318.9139,0.0967,0.0955
SD,nan,1.0671,0.8020,5291.9063,5291.9063,0.0421,0.0384


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## dt_cds_dt

In [ ]:

dt_cds_dt = exp.create_model('dt_cds_dt')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = dt_cds_dt, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,1.9228,1.4530,8693.4392,8693.4392,0.0624,0.0644
1,2021.0000,0.4076,0.3069,1935.8002,1935.8002,0.0154,0.0153
2,2022.0000,5.6951,4.2918,28195.4860,28195.4860,0.1824,0.2007
Mean,nan,2.6752,2.0172,12941.5752,12941.5752,0.0867,0.0934
SD,nan,2.2232,1.6751,11133.3663,11133.3663,0.0703,0.0784


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## ALL GRAPHS

In [ ]:
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = ada_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})              

NameError: name 'theta' is not defined

## Avaliando a Produtividade

In [ ]:
#função para tratar campo data                                                                                                                                                                      
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('dbsoja_br.xls', sheet_name='Planilha1')
print(df.head(10))

   date  producao  produtividade     area
0  1977   12145.0           1748   6949.0
1  1978    9726.0           1250   7780.0
2  1979   10200.0           1251   8151.0
3  1980   14887.4           1700   8755.9
4  1981   15484.8           1781   8693.4
5  1982   12890.9           1536   8393.2
6  1983   14532.9           1728   8412.0
7  1984   15340.5           1674   9162.9
8  1985   18211.5           1808  10074.0
9  1986   13207.5           1369   9644.4


In [ ]:
df.drop(['producao','area'], axis = 1,inplace=True)
df.head(10)

,date,produtividade
0,1977,1748
1,1978,1250
2,1979,1251
3,1980,1700
4,1981,1781
5,1982,1536
6,1983,1728
7,1984,1674
8,1985,1808
9,1986,1369


In [ ]:
df.set_index('date',inplace=True)

In [ ]:
df=df.sort_index(axis=0)
print(df.head(10))

      produtividade
date               
1977           1748
1978           1250
1979           1251
1980           1700
1981           1781
1982           1536
1983           1728
1984           1674
1985           1808
1986           1369


In [ ]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=15, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,15
1,Target,produtividade
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(48, 1)"
5,Transformed data shape,"(48, 1)"
6,Transformed train set shape,"(47, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [ ]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})


In [ ]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})


In [ ]:
top4 = exp.compare_models(n_select = 4)


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
exp_smooth,Exponential Smoothing,1.0525,0.8429,217.6841,217.6841,0.0692,0.0656,0.0367
polytrend,Polynomial Trend Forecaster,1.0541,0.8441,218.4752,218.4752,0.0679,0.0661,0.0200
ets,ETS,1.0541,0.8441,218.4700,218.4700,0.0679,0.0661,0.0367
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,1.0681,0.8553,221.4635,221.4635,0.0686,0.0670,0.0600
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,1.0733,0.8595,222.4587,222.4587,0.0692,0.0672,0.0500
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,1.0857,0.8695,225.1935,225.1935,0.0697,0.0680,0.2100
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,1.1118,0.8903,230.6386,230.6386,0.0714,0.0696,0.0567
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,1.1118,0.8903,230.6407,230.6407,0.0714,0.0696,0.0467
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,1.1118,0.8903,230.6407,230.6407,0.0714,0.0696,0.0467
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,1.1118,0.8903,230.6367,230.6367,0.0714,0.0696,0.0767


Processing:   0%|          | 0/112 [00:00<?, ?it/s]

## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data.
 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

In [ ]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":50})


####  exp_smooth

In [ ]:
exp_smooth= exp.create_model('exp_smooth')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 7},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(exp_smooth, fh = 7)

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2024,3501.8683
2025,3549.7717
2026,3597.6751
2027,3645.5785
2028,3693.4819
2029,3741.3854


In [ ]:
polytrend = exp.create_model('polytrend')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = polytrend, plot = 'forecast', 
               data_kwargs = {'fh' : 6},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(polytrend, fh = 6)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,0.9044,0.7235,187.5116,187.5116,0.0532,0.0546
1,2021.0000,1.8171,1.4553,374.2414,374.2414,0.1237,0.1165
2,2022.0000,0.4408,0.3535,93.6725,93.6725,0.0267,0.0271
Mean,nan,1.0541,0.8441,218.4752,218.4752,0.0679,0.0661
SD,nan,0.5717,0.4578,116.6156,116.6156,0.0409,0.0374


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2024,3465.8899
2025,3510.7293
2026,3555.5687
2027,3600.4081
2028,3645.2475
2029,3690.0868


#### polytrend

In [ ]:
ets = exp.create_model('ets')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 6},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,0.9044,0.7235,187.5095,187.5095,0.0532,0.0546
1,2021.0000,1.8170,1.4553,374.2280,374.2280,0.1237,0.1165
2,2022.0000,0.4408,0.3535,93.6727,93.6727,0.0267,0.0271
Mean,nan,1.0541,0.8441,218.4700,218.4700,0.0679,0.0661
SD,nan,0.5717,0.4578,116.6097,116.6097,0.0409,0.0374


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


### xgboost

In [ ]:
xgboost_cds_dt = exp.create_model('xgboost_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = xgboost_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 7},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(xgboost_cds_dt, fh = 7)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,0.4865,0.3892,100.8726,100.8726,0.0286,0.0282
1,2021.0000,3.0348,2.4306,625.0397,625.0397,0.2066,0.1872
2,2022.0000,1.5498,1.2428,329.3124,329.3124,0.0939,0.0985
Mean,nan,1.6904,1.3542,351.7416,351.7416,0.1097,0.1047
SD,nan,1.0451,0.8371,214.5772,214.5772,0.0735,0.0651


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2024,3755.6506
2025,3272.1674
2026,3674.7874
2027,3666.9899
2028,3482.5761
2029,3654.1867
2030,3777.3762


### ALL GRAPHS

In [ ]:
exp.plot_model(estimator = lightgbm_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = xgboost_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = polytrend, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

In [ ]:
## Avaliando a Area

In [ ]:
#função para tratar campo data                                                                                                                                                                      
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('dbsoja_br.xls', sheet_name='Planilha1')
print(df.head(10))

   date  producao  produtividade     area
0  1977   12145.0           1748   6949.0
1  1978    9726.0           1250   7780.0
2  1979   10200.0           1251   8151.0
3  1980   14887.4           1700   8755.9
4  1981   15484.8           1781   8693.4
5  1982   12890.9           1536   8393.2
6  1983   14532.9           1728   8412.0
7  1984   15340.5           1674   9162.9
8  1985   18211.5           1808  10074.0
9  1986   13207.5           1369   9644.4


In [ ]:
df.drop(['producao','produtividade'], axis = 1,inplace=True)
df.set_index('date',inplace=True)
df=df.sort_index(axis=0)
print(df.head(10))

         area
date         
1977   6949.0
1978   7780.0
1979   8151.0
1980   8755.9
1981   8693.4
1982   8393.2
1983   8412.0
1984   9162.9
1985  10074.0
1986   9644.4


In [ ]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=15, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,15
1,Target,area
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(48, 1)"
5,Transformed data shape,"(48, 1)"
6,Transformed train set shape,"(47, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [ ]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

In [ ]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

In [ ]:
top4 = exp.compare_models(n_select = 4)


## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data.
 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.


In [ ]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":50})

In [ ]:
exp_smooth = exp.create_model('exp_smooth')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 6},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(exp_smooth, fh = 6)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,1.4474,1.1790,1526.7334,1526.7334,0.0386,0.0394
1,2021.0000,0.6330,0.5154,689.6202,689.6202,0.0166,0.0168
2,2022.0000,1.1108,0.9091,1231.7055,1231.7055,0.0279,0.0283
Mean,nan,1.0637,0.8678,1149.3530,1149.3530,0.0277,0.0282
SD,nan,0.3342,0.2725,346.6757,346.6757,0.0090,0.0092


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2024,45665.0977
2025,47255.9216
2026,48846.7455
2027,50437.5695
2028,52028.3934
2029,53619.2173


In [ ]:
ets = exp.create_model('ets')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 7},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(ets, fh = 7)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,1.4225,1.1587,1500.4454,1500.4454,0.0380,0.0387
1,2021.0000,0.6292,0.5123,685.4829,685.4829,0.0165,0.0167
2,2022.0000,1.0892,0.8914,1207.7889,1207.7889,0.0274,0.0278
Mean,nan,1.0469,0.8542,1131.2391,1131.2391,0.0273,0.0277
SD,nan,0.3252,0.2652,337.0815,337.0815,0.0088,0.0090


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2024,45666.5912
2025,47253.1990
2026,48839.8067
2027,50426.4145
2028,52013.0222
2029,53599.6300
2030,55186.2377


In [ ]:
auto_arima= exp.create_model('auto_arima')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = auto_arima, plot = 'forecast', 
               data_kwargs = {'fh' : 7},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(auto_arima, fh = 7)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,1.6708,1.3610,1762.4071,1762.4071,0.0446,0.0456
1,2021.0000,0.5697,0.4639,620.7000,620.7000,0.0150,0.0148
2,2022.0000,0.5657,0.4630,627.3000,627.3000,0.0142,0.0143
Mean,nan,0.9354,0.7626,1003.4690,1003.4690,0.0246,0.0249
SD,nan,0.5200,0.4231,536.6570,536.6570,0.0141,0.0146


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2024,46172.6751
2025,47658.2998
2026,49143.9245
2027,50629.5493
2028,52115.1740
2029,53600.7987
2030,55086.4234
